In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler # https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

# metrics: https://scikit-learn.org/stable/modules/model_evaluation.html

In [2]:
test = pd.read_csv('./test.csv')
dT = pd.read_csv('./train.csv')
dT.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
0,0,0.10859,0.004314,-37.566,0.017364,0.28915,-10.25100,135.12,168900.0,3.992400e+14,...,-12.2280,1.7482,1.90960,-7.11570,4378.80,1.2096,8.613400e+14,140.1,1.01770,1
1,1,0.10090,0.299610,11822.000,0.276500,0.45970,-0.83733,1721.90,119810.0,3.874100e+15,...,-56.7580,4.1684,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.0,0.28359,0
2,2,0.17803,-0.006980,907.270,0.272140,0.45948,0.17327,2298.00,360650.0,1.224500e+13,...,-5.7688,1.2042,0.26290,8.13120,45119.00,1.1764,3.218100e+14,3838.2,0.40690,1
3,3,0.15236,0.007259,780.100,0.025179,0.51947,7.49140,112.51,259490.0,7.781400e+13,...,-34.8580,2.0694,0.79631,-16.33600,4952.40,1.1784,4.533000e+12,4889.1,0.51486,1
4,4,0.11623,0.502900,-109.150,0.297910,0.34490,-0.40932,2538.90,65332.0,1.907200e+15,...,-13.6410,1.5298,1.14640,-0.43124,3856.50,1.4830,-8.991300e+12,NaN,0.23049,1


In [3]:
dT.max() # lots of different amplitudes, maybe minmax
print(dT.isnull().sum())

id           0
f1       15247
f2       15190
f3       15491
f4       15560
         ...  
f115     15559
f116     15589
f117     15407
f118     15212
claim        0
Length: 120, dtype: int64


In [4]:
# Só o básico:
train = dT.dropna().copy()
print(train.isnull().sum())

# matrix = np.triu(dT.corr())
# sns.heatmap(annot=True,data=dT.corr(), fmt='.1g',square=True,mask=matrix) # Enorme não da pra ver nada

id       0
f1       0
f2       0
f3       0
f4       0
        ..
f115     0
f116     0
f117     0
f118     0
claim    0
Length: 120, dtype: int64


In [9]:
# Deixar pronto pra treinar:

y = train['claim'].copy()
X = train.drop('claim', axis=1)

# NÚMERO DESAJADO (MÁXIMO) DE FEATURES:
# num_feats = int(len(X.columns)/6)+1  # Chute: diminuir pra 20 (número razoavel né sei lá) => Alguns alg. não convergiram!
num_feats = int(len(X.columns)/4)+1  # Chute: diminuir pra 20 (número razoavel né sei lá) => Alguns alg. não convergiram!
print(num_feats) 

print(y.head()) # olhando target (binário => classificação!)


20
1     0
8     0
9     0
11    0
14    1
Name: claim, dtype: int64


In [ ]:
# Libs pra rodar a próx. seção
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

In [15]:
# Dealing with lots of features: https://towardsdatascience.com/the-5-feature-selection-algorithms-every-data-scientist-need-to-know-3a6b566efd2
def cor_selector(X, y,num_feats):
    cor_list = []
    feature_name = X.columns.tolist()
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature

# Dryrun: para 20 features, demorou uns 
def selector(X,y,num_feats):
    feature_name = X.columns.tolist()
    
    # Pearson Cor.
    print('~ PearsonCor:vanilla ~')
    cor_support, cor_feature = cor_selector(X, y,num_feats)
    print(str(len(cor_feature)), 'selected features')

    # Chi-quadrado
    print('~ Chi-Sqrd:vanilla ~')
    X_norm = MinMaxScaler().fit_transform(X)
    chi_selector = SelectKBest(chi2, k=num_feats)
    chi_selector.fit(X_norm, y)
    chi_support = chi_selector.get_support()
    chi_feature = X.loc[:,chi_support].columns.tolist()
    print(str(len(chi_feature)), 'selected features')

    # Elim. recurs.
    print('~ RecElim:vanilla ~')
    rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=num_feats, step=10, verbose=5)
    rfe_selector.fit(X_norm, y)
    rfe_support = rfe_selector.get_support()
    rfe_feature = X.loc[:,rfe_support].columns.tolist()
    print(str(len(rfe_feature)), 'selected features')

    # Lasso: SelectFromModel
    print('~ Lasso:vanilla ~')
    embeded_lr_selector = SelectFromModel(LogisticRegression(penalty="l2"), max_features=num_feats)
    embeded_lr_selector.fit(X_norm, y)

    embeded_lr_support = embeded_lr_selector.get_support()
    embeded_lr_feature = X.loc[:,embeded_lr_support].columns.tolist()
    print(str(len(embeded_lr_feature)), 'selected features')

    # Tree-based: SelectFromModel
    # Vanilla:
    print('~ Tree:vanilla ~')
    embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=num_feats)
    embeded_rf_selector.fit(X, y)

    embeded_rf_support = embeded_rf_selector.get_support()
    embeded_rf_feature = X.loc[:,embeded_rf_support].columns.tolist()
    print(str(len(embeded_rf_feature)), 'selected features')

    # lightgbm
    print('~ Tree:lgbm ~')
    lgbc=LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
                reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)

    embeded_lgb_selector = SelectFromModel(lgbc, max_features=num_feats)
    embeded_lgb_selector.fit(X, y)

    embeded_lgb_support = embeded_lgb_selector.get_support()
    embeded_lgb_feature = X.loc[:,embeded_lgb_support].columns.tolist()
    print(str(len(embeded_lgb_feature)), 'selected features')

    # //TODO: implementar xgboost similar ao codigo acima (marretinha)
    # //DONE: transformar em função para testar resultados em função de num. de features 

    feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embeded_lr_support,
                                    'Random Forest':embeded_rf_support, 'LightGBM':embeded_lgb_support})
    # count the selected times for each feature
    feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
    # display the top 100
    feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)

    return feature_selection_df

In [17]:
def showBest(feature_selection_df):
    feature_selection_df.index = range(1, len(feature_selection_df)+1)
    feature_selection_df.head(num_feats)

# meu chute enquanto vou vacinar:
chute = [5, 10, 15, 20, 25, 30, 40, 42, 50, 60, 70, 80]
dfs = []
for num_feats in chute:
    print(num_feats)
    feature_selection_df = selector(X,y,num_feats)
    dfs.append(feature_selection_df)

5
~ PearsonCor:vanilla ~
5 selected features
~ Chi-Sqrd:vanilla ~
